# Residuals Analysis

In [1]:
import numpy as np
import pandas as pd
from statsmodels import regression
import statsmodels.api as sm
import statsmodels.stats.diagnostic as smd
import scipy.stats as stats
import matplotlib.pyplot as plt
import math

import yfinance as yf
import datetime as dt

from jupyterthemes import jtplot
jtplot.style(theme='onedork', figsize=(15, 9))

In [ ]:
class ResidualAnalysis():
    def __init__(self, asset, benchmark, start, end):
        self.asset = asset
        self.benchmark = benchmark
        self.start = start
        self.end = end
    
    def get_data(self):
        asset = yf.download(self.asset, start=self.start, end=self.end)
        benchmark = yf.download(self.benchmark, start=self.start, end=self.end)
        
        self.asset_returns = np.log(asset['Close'].div(asset['Close'].shift(1)))
        self.benchmark_returns = np.log(benchmark['Close'].div(benchmark['Close'].shift(1)))
    
    def linear_regression_model(self):
        self.benchmark_returns = sm.add_constant(self.benchmark_returns)
        self.model = sm.OLS(self.asset_returns, self.benchmark_returns).fit()
        self.benchmark_returns = self.benchmark_returns[:, 1]
        B0, B1 = self.model.params
    
    def heteroscedasticity_test(self):
        bp_test = smd.het_breuschpagan(self.model.resid, self.model.model.exog)

        print(f'Lagrange Multiplier Statistic: {bp_test[0]}')
        print(f'P-value: {bp_test[1]}')
        print(f'f-value: {bp_test[2]}')
        print(f'f_p-value: {bp_test[3]}\n')
        if bp_test[1] > 0.05:
            print('The relationship is not heteroscedastic.')
        if bp_test[1] < 0.05:
            print('The relationship is heteroscedastic.')
    
    def autocorrelation_test(self):
        ljung_box = smd.acorr_ljungbox(self.asset_returns)
        print(f'P-Values: {ljung_box[1]}\n')
        if any(ljung_box[1] < 0.05):
            print('The residuals are autocorrelated.')
        else:
            print('The residuals are not autocorrelated.')
    
    
    